In [1]:
!pip install deepctr

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# module & data import

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing.sequence import pad_sequences
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names
from deepctr.models.xdeepfm import xDeepFM
from sklearn.model_selection import train_test_split

# importing libraries
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

from functools import reduce
import operator
from collections import Counter

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [3]:
category_vector = pd.read_csv('category_vector.csv') # doc2vec embedding
vector = pd.read_csv("vectors.csv", encoding = 'utf8') # 키워드 vector
bert_df = pd.read_csv("isbn_title.csv", encoding = 'cp949')# all data import
all_book = pd.read_csv('한얼님께받은전체데이터.csv')

In [4]:
univlog_1 = pd.read_csv('univlog_1.csv')
univlog_2 = pd.read_csv('univlog_2.csv')
univlog_3 = pd.read_csv('univlog_3.csv')

In [67]:
# all data import
gnb = pd.read_csv('gnbgnb.csv')

In [68]:
univ = univlog_1 ['company_name'][0] #univ_name choose

In [69]:
data = univlog_1

In [70]:
data

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,label,title,author,publisher,company_name
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,0.0,성경(46판 2단 금장색인) (BB6 BH) (250130),31736,2394,University of Michigan
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,종교,불교,불교철학사상,불교철학사상,이지관가산문고,0.0,남북전육부 율장비교연구(3판),184277,2326,University of Michigan
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,0.0,제7감각을 기르자,213344,17073,University of Michigan
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,1.0,디오니소스의 노동 1,143079,2428,University of Michigan
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,시에세이,한국시,현대시,현대시,표성배갈무리,1.0,아침햇살이 그립다,238012,2428,University of Michigan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977140,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,요리,와인커피음료,와인,와인,이동현김정,0.0,즐거운 와인 여행 2(포켓북(문고판)),174715,4055,University of Michigan
977141,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,만화,그래픽노블,판타지,판타지,애니북스,0.0,스타워즈 STAR WARS 클론워즈 어드벤처 5,5326,14828,University of Michigan
977142,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,만화,웹툰카툰에세이,일상생활드라마,일상생활드라마,쿠루네코 야마토중앙북스,0.0,쿠루네코 1(반양장),230136,19582,University of Michigan
977143,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,만화,일상생활드라마가족만화,일상생활드라마가족만화,일상생활드라마가족만화,편집부삼성출판사,0.0,거침없이 하이킥 2(완결)(MBC 시트콤 사진만화),237109,11357,University of Michigan


# PCA 차원 축소 진행

In [71]:
# 차원축소할 data
dense_features = ['catev_' + str(i) for i in range(1,80)]
dense_features2 = [str(i) for i in range(1,80)]
dense_features.extend(dense_features2)

In [72]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# PCA 적용 - 컴포넌트는 1로 설정합니다.
df_scaled = StandardScaler().fit_transform(data[dense_features])


pca = PCA(n_components=5)
#fit( )과 transform( ) 을 호출하여 PCA 변환 데이터 반환
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)
print(df_pca.shape)


(977145, 5)


In [73]:
df = pd.DataFrame(df_pca,columns=['pca_vector' + str(i) for i in range(1,6)]) # 차원축소 dataframe생성

In [74]:
tmpcol =  ["title","isbn","author", "publisher","date",'label','Cn_1','Cn_2','Cn_3','Cn_4']
df[tmpcol] = data[tmpcol]

In [75]:
# categorising the features into sparse/dense feaure set
sparse_features = ["isbn","author", "publisher","date"]
dense_features = ['pca_vector' + str(i) for i in range(1,6)]

target=['label']

# data inputation for missing values
data[sparse_features] = data[sparse_features].fillna('-1',)

# encoding function  // labelencoder ; 문자를 0부터 증가하는 정수형 숫자로 바꿔주는기능을 제공
def encoding(data,feat,encoder):
    data[feat] = encoder.fit_transform(data[feat])
    
# encoding for categorical feautures
[encoding(data, feat, LabelEncoder()) for feat in sparse_features]

# 1.Label Encoding for sparse features
for feat in sparse_features:
  lbe = LabelEncoder()
  data[feat] = lbe.fit_transform(data[feat])



In [76]:
df # 최종 input df

,pca_vector1,pca_vector2,pca_vector3,pca_vector4,pca_vector5,title,isbn,author,publisher,date,label,Cn_1,Cn_2,Cn_3,Cn_4
0,0.453872,4.818304,-0.974435,-1.625968,-2.539101,성경(46판 2단 금장색인) (BB6 BH) (250130),177,31736,2394,44,0.0,종교,가톨릭,가톨릭성서,가톨릭성서
1,-2.868274,4.525505,0.639141,-1.557306,-2.093210,남북전육부 율장비교연구(3판),138,184277,2326,14,0.0,종교,불교,불교철학사상,불교철학사상
2,4.421445,1.429144,-2.353265,4.170881,0.237731,제7감각을 기르자,120,213344,17073,32,0.0,인문,교육학,교수학습법,학습법
3,1.055648,-2.192316,1.126266,3.244836,0.589466,디오니소스의 노동 1,178,143079,2428,34,1.0,정치사회,사회문제복지,노동문제,노동문제일반
4,0.363275,0.537515,1.713908,1.914327,2.455601,아침햇살이 그립다,179,238012,2428,39,1.0,시에세이,한국시,현대시,현대시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977140,-0.493753,0.481390,1.345457,-0.716148,-1.681578,즐거운 와인 여행 2(포켓북(문고판)),360444,174715,4055,46,0.0,요리,와인커피음료,와인,와인
977141,2.606939,-2.719278,-1.356452,-2.149050,-3.172841,스타워즈 STAR WARS 클론워즈 어드벤처 5,320997,5326,14828,46,0.0,만화,그래픽노블,판타지,판타지
977142,0.792029,0.553976,1.079489,-0.843891,-1.077683,쿠루네코 1(반양장),359064,230136,19582,46,0.0,만화,웹툰카툰에세이,일상생활드라마,일상생활드라마
977143,1.986904,0.000053,0.976288,0.639999,-1.735987,거침없이 하이킥 2(완결)(MBC 시트콤 사진만화),96910,237109,11357,46,0.0,만화,일상생활드라마가족만화,일상생활드라마가족만화,일상생활드라마가족만화


In [77]:
data = df

# 알고리즘

In [78]:
# creating a 4 bit embedding for every sparse feature
sparse_feature_columns = [SparseFeat(feat,vocabulary_size = data[feat].nunique(), embedding_dim=4)
                         for i,feat in enumerate(sparse_features)]

# Creating a dense feat
dense_feature_columns = [DenseFeat(feat,1) for feat in dense_features]

# features to be used for dnn part of xdeepfm
dnn_feature_columns = sparse_feature_columns + dense_feature_columns
# features to be used for linear part of xdeepfm
linear_feature_columns = sparse_feature_columns + dense_feature_columns

# feature names
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [79]:
feature_names

['isbn',
 'author',
 'publisher',
 'date',
 'pca_vector1',
 'pca_vector2',
 'pca_vector3',
 'pca_vector4',
 'pca_vector5']

In [80]:


# creating train test splits
train, test = train_test_split(data, test_size = 0.2) # 200*0.2 = 40개를 test_dataset으로 분리
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}



In [81]:


model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),
cin_layer_size=(128, 128),
cin_split_half=True, cin_activation='relu'
,l2_reg_linear=1e-05,
l2_reg_embedding=1e-05, l2_reg_dnn=0, l2_reg_cin=0,
seed=1024, dnn_dropout=0,dnn_activation='relu',
dnn_use_bn=False, task='binary')



# 학습 진행

In [82]:

# compiling the model
import tensorflow as tf

# compiling the model
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])


In [83]:
# training the model
history = model.fit(train_model_input, train[target].values,
                    batch_size = 256, epochs = 10, verbose = 2, validation_split = 0.2, )

Epoch 1/10
2443/2443 - 15s - loss: 0.4022 - binary_crossentropy: 0.3932 - precision_1: 0.7072 - recall_1: 0.6313 - auc_1: 0.8487 - val_loss: 0.3678 - val_binary_crossentropy: 0.3483 - val_precision_1: 0.7499 - val_recall_1: 0.7163 - val_auc_1: 0.8937
Epoch 2/10
2443/2443 - 13s - loss: 0.0420 - binary_crossentropy: 0.0260 - precision_1: 0.8165 - recall_1: 0.7911 - auc_1: 0.9399 - val_loss: 0.4009 - val_binary_crossentropy: 0.3872 - val_precision_1: 0.8541 - val_recall_1: 0.8337 - val_auc_1: 0.9620
Epoch 3/10
2443/2443 - 13s - loss: 0.0113 - binary_crossentropy: 0.0017 - precision_1: 0.8740 - recall_1: 0.8563 - auc_1: 0.9710 - val_loss: 0.5251 - val_binary_crossentropy: 0.5175 - val_precision_1: 0.8899 - val_recall_1: 0.8717 - val_auc_1: 0.9765
Epoch 4/10
2443/2443 - 13s - loss: 0.0264 - binary_crossentropy: 0.0182 - precision_1: 0.8995 - recall_1: 0.8807 - auc_1: 0.9791 - val_loss: 0.4622 - val_binary_crossentropy: 0.4503 - val_precision_1: 0.9072 - val_recall_1: 0.8856 - val_auc_1: 0.9

In [84]:
#predicting
pred_ans_xdeep_test = model.predict(test_model_input, batch_size=256)

In [85]:
#predicting
pred_ans_xdeep_train = model.predict(train_model_input, batch_size=256)

In [86]:
test['pred_ctr'] = pred_ans_xdeep_test
train['pred_ctr'] = pred_ans_xdeep_train

In [87]:
ctr = pd.concat([train,test])
ctr # final data

,pca_vector1,pca_vector2,pca_vector3,pca_vector4,pca_vector5,title,isbn,author,publisher,date,label,Cn_1,Cn_2,Cn_3,Cn_4,pred_ctr
574908,-0.265271,-1.410594,-0.194775,-2.355456,1.162284,릴케의 베네치아 여행,906272,80622,8705,55,0.0,시에세이,나라별 에세이,독일에세이,독일에세이,0.000000e+00
54711,-1.497945,0.127244,4.153764,1.687951,1.242311,영어 팡팡 스티커(EQ를 높여주는 사진 스티커 200장),126885,146730,14864,47,0.0,유아세,유아놀이,스티커북색칠하기,스티커북색칠하기,0.000000e+00
822932,2.814482,-2.430508,4.514048,-1.850965,-2.597043,아이큐 삐삐(V.T:2),15420,237135,13462,36,0.0,유아세,유아그림책,세계명작그림책,세계명작그림책,0.000000e+00
686134,-1.053154,-0.695058,2.166136,1.791717,-0.434435,주택도시보증공사 필기전형(2021)(NCS),705750,226077,12077,59,0.0,취업수험서,인적성직무능력,공기업공사공무원,공기업공사공무원,9.800818e-17
524986,-3.372234,3.397987,-0.999139,1.897480,-1.577212,요괴: 또 하나의 일본의 문화코드(한양대 일본학국제비교연구소 비교일본학 총서 2)(...,824850,247217,15740,57,1.0,역사문화,동양사,일본사,일본문화사,8.529519e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61700,-6.586243,0.996677,-1.081669,-0.489416,-2.061818,일본 문학사(한권으로 읽는),138477,99947,24167,51,1.0,인문,세계문학론,일본문학이론,일본문학이론,8.948886e-01
876204,-4.280053,1.549791,0.766555,-3.610937,-3.145687,잠자는 숲 속의 공주(Sleeping Beauty)(느낌표 세계 명작동화 영어와 한...,601377,120483,14358,59,0.0,유아세,유아그림책,세계명작그림책,세계명작그림책,6.137588e-02
894094,-2.369306,1.204472,-1.230123,3.321152,0.232473,문화콘텐츠와 선택의 동기(큰글씨책)(커뮤니케이션이해총서),712662,60885,21046,60,0.0,정치사회,사회학,각국사회문화,문화일반,9.638471e-01
320836,0.443096,1.664730,-0.408722,-0.500450,-2.036110,나는 예수입니다,517223,58126,21864,58,1.0,종교,종교일반,종교이야기,종교이야기,9.500925e-01


In [88]:
ctr_sorted = ctr.sort_values(by=ctr.columns[-1],ascending=False)

In [89]:
ctr_sorted

,pca_vector1,pca_vector2,pca_vector3,pca_vector4,pca_vector5,title,isbn,author,publisher,date,label,Cn_1,Cn_2,Cn_3,Cn_4,pred_ctr
392430,1.755371,2.355345,-0.963709,1.017794,0.209271,동시대(제51호),629361,79237,16993,49,1.0,시에세이,한국시,현대시,현대시,1.0
63597,-0.663388,-2.685909,0.800976,0.359834,0.145431,고교교사,140842,24099,25082,50,1.0,만화,만화,만화,만화,1.0
969937,4.511264,-2.721269,-0.693881,3.404626,-3.948121,중사 클리든 1,115845,226067,7384,45,1.0,소설,한국소설,판타지소설,판타지소설,1.0
98096,2.301877,-0.870869,1.170021,0.824579,-3.103971,준비됐지(창비아동문고 247),187292,57706,20211,47,1.0,어린이초등,어린이문학,동화책,외국작가,1.0
955068,3.863379,-1.807806,1.181370,0.125904,-0.773449,페스티벌 컬렉션(The Festival Collection) 6,370097,20581,8793,48,1.0,예술대중문화,음악,건반악기,어린이 피아노교본,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708475,4.410362,1.491379,-1.089933,2.573039,-0.048739,WRITING PRACTICE GRADE. 3,69735,237109,1772,45,0.0,외국어,어린이영어,작문쓰기,작문쓰기,0.0
822623,0.123570,-3.672561,-3.715044,-3.757346,-1.576827,달님이 본 것은(세계 걸작 그림책 지크),208930,115776,9910,34,0.0,유아세,유아그림책,세계명작그림책,세계명작그림책,0.0
720650,5.675230,-1.608537,-1.384931,0.386181,5.943196,"Wonder of Bald Eagles, The",71143,7578,21121,43,0.0,외국어,어린이영어,동화그림책리더스,리더스북,0.0
701430,4.258456,-1.647398,-1.471439,3.641147,0.867519,Spectrum Vocabulary Grade. 4 (Updated),74492,17904,1359,45,0.0,외국어,어린이영어,유아영어,알파벳단어,0.0


# user's favorite cn3 filtering

# 가장 관심있는 Cn찾기

In [91]:
# gnb에서 검색 로그찾기
gnb_univ = gnb[gnb['company_name'].str.contains(univ)]
user_log_cn3 = gnb_univ['Cn_3'].values.tolist() # user_log_cn3 ; user가 검색한 책들의 Cn3_list
counter = Counter(user_log_cn3)
keycount = dict(counter)
sorted_cn3 = sorted(keycount.items(), reverse=True,key=lambda item: item[1])
cn3_user_interestd = dict(sorted_cn3[:3])
cn3_user_interestd # 유저가 가장 관심있는 카테고리

{'한국소설일반': 209, '한국에세이': 120, '한국문학이론': 76}

In [92]:
userlike = ctr_sorted['Cn_3'].isin(cn3_user_interestd)
like = ctr_sorted[userlike]

In [104]:
like = like[:30]

In [105]:
like.to_csv("univ1_recommend.csv",index=False)